# Getting started with MIPLearn

## Introduction

**MIPLearn** is an open source framework that uses machine learning (ML) to accelerate the performance of both commercial and open source mixed-integer programming solvers (e.g. Gurobi, CPLEX, XPRESS, Cbc or SCIP). In this tutorial, we will:

1. Install the Julia/JuMP version of MIPLearn
2. Model a simple optimization problem using JuMP
3. Generate training data and train the ML models
4. Use the ML models together with SCIP to solve new instances

<div class="alert alert-info">
Note
    
In this tutorial, we use SCIP because it is more widely available than commercial MIP solvers. However, all the steps below should work for Gurobi, CPLEX or XPRESS, as long as you have a license for these solvers. The performance impact of MIPLearn may also change for different solvers.
</div>

<div class="alert alert-warning">
Warning
    
MIPLearn is still in early development stage. If run into any bugs or issues, please submit a bug report in our GitHub repository. Comments, suggestions and pull requests are also very welcome!
    
</div>


## Installation

MIPLearn is available in two versions:

- Python version, compatible with the Pyomo modeling language,
- Julia version, compatible with the JuMP modeling language.

In this tutorial, we will demonstrate how to use and install the Julia/JuMP version of the package. The first step is to install the Julia programming language in your computer. [See the official instructions for more details](https://julialang.org/downloads/). Note that MIPLearn was developed and tested with Julia 1.6, and may not be compatible with newer versions of the language. After Julia is installed, launch its console and run the following commands to download and install the package:

In [1]:
using Pkg
Pkg.add(PackageSpec(url="https://github.com/ANL-CEEESA/MIPLearn.jl.git"))

    Updating git-repo `https://github.com/ANL-CEEESA/MIPLearn.jl.git`
    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
    Updating `~/Packages/MIPLearn/dev/docs/jump-tutorials/Project.toml`
  [2b1277c3] ~ MIPLearn v0.2.0 `https://github.com/ANL-CEEESA/MIPLearn.jl.git#dev` ⇒ v0.2.0 `https://github.com/ANL-CEEESA/MIPLearn.jl.git#dev`
    Updating `~/Packages/MIPLearn/dev/docs/jump-tutorials/Manifest.toml`
  [2b1277c3] ~ MIPLearn v0.2.0 `https://github.com/ANL-CEEESA/MIPLearn.jl.git#dev` ⇒ v0.2.0 `https://github.com/ANL-CEEESA/MIPLearn.jl.git#dev`
    Building MIPLearn → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/8e595dec310860a52e01c8875f51165fcccb1875/build.log`
Precompiling project...
  ✓ MIPLearn
1 dependency successfully precompiled in 10 seconds (96 already precompiled)


In addition to MIPLearn itself, we will also install a few other packages that are required for this tutorial:

- [**SCIP**](https://www.scipopt.org/), one of the fastest non-commercial MIP solvers currently available
- [**JuMP**](https://jump.dev/), an open source modeling language for Julia
- [**Distributions.jl**](https://github.com/JuliaStats/Distributions.jl), a statistics package that we will use to generate random inputs
- [**Glob.jl**](https://github.com/vtjnash/Glob.jl), a package that retrieves all files in a directory matching a certain pattern

In [2]:
using Pkg
Pkg.add([
    PackageSpec(url="https://github.com/scipopt/SCIP.jl.git", rev="7aa79aaa"),
    PackageSpec(name="JuMP", version="0.21"),
    PackageSpec(name="Distributions", version="0.25"),
    PackageSpec(name="Glob", version="1"),
])

   Resolving package versions...
  No Changes to `~/Packages/MIPLearn/dev/docs/jump-tutorials/Project.toml`
  No Changes to `~/Packages/MIPLearn/dev/docs/jump-tutorials/Manifest.toml`


<div class="alert alert-info">
    
Note
    
In the code above, we install specific version of all packages to ensure that this tutorial keeps running in the future, even when newer (and possibly incompatible) versions of the packages are released. This is usually a recommended practice for all Julia projects.
    
</div>

## Modeling a simple optimization problem

To illustrate how can MIPLearn be used, we will model and solve a small optimization problem related to power systems optimization. The problem we discuss below is a simplification of the **unit commitment problem,** a practical optimization problem solved daily by electric grid operators around the world. 

Suppose that you work at a utility company, and that it is your job to decide which electrical generators should be online at a certain hour of the day, as well as how much power should each generator produce. More specifically, assume that your company owns $n$ generators, denoted by $g_1, \ldots, g_n$. Each generator can either be online or offline. An online generator $g_i$ can produce between $p^\text{min}_i$ to $p^\text{max}_i$ megawatts of power, and it costs your company $c^\text{fix}_i + c^\text{var}_i y_i$, where $y_i$ is the amount of power produced. An offline generator produces nothing and costs nothing. You also know that the total amount of power to be produced needs to be exactly equal to the total demand $d$ (in megawatts). To minimize the costs to your company, which generators should be online, and how much power should they produce?

This simple problem can be modeled as a *mixed-integer linear optimization* problem as follows. For each generator $g_i$, let $x_i \in \{0,1\}$ be a decision variable indicating whether $g_i$ is online, and let $y_i \geq 0$ be a decision variable indicating how much power does $g_i$ produce. The problem is then given by:

$$
\begin{align}
\text{minimize } \quad & \sum_{i=1}^n \left( c^\text{fix}_i x_i + c^\text{var}_i y_i \right) \\
\text{subject to } \quad & y_i \leq p^\text{max}_i x_i & i=1,\ldots,n \\
& y_i \geq p^\text{min}_i x_i & i=1,\ldots,n \\
& \sum_{i=1}^n y_i = d \\
& x_i \in \{0,1\} & i=1,\ldots,n \\
& y_i \geq 0 & i=1,\ldots,n
\end{align}
$$

<div class="alert alert-info">
    
Note
    
We use a simplified version of the unit commitment problem in this tutorial just to make it easier to follow. MIPLearn can also handle realistic, large-scale versions of this problem. See benchmarks for more details.
    
</div>

Next, let us convert this abstract mathematical formulation into a concrete optimization model, using Julia and JuMP. We start by defining a data structure that holds all the input data.

In [3]:
Base.@kwdef struct UnitCommitmentData
    demand::Float64
    pmin::Vector{Float64}
    pmax::Vector{Float64}
    cfix::Vector{Float64}
    cvar::Vector{Float64}
end;

Next, we create a function that converts this data structure into a concrete JuMP model. For more details on the JuMP syntax, see [the official JuMP documentation](https://jump.dev/JuMP.jl/stable/).

In [4]:
using JuMP

function build_uc_model(data::UnitCommitmentData)::Model
    model = Model()
    n = length(data.pmin)
    @variable(model, x[1:n], Bin)
    @variable(model, y[1:n] >= 0)
    @objective(
        model,
        Min,
        sum(
            data.cfix[i] * x[i] +
            data.cvar[i] * y[i]
            for i in 1:n
        )
    )
    @constraint(model, eq_max_power[i in 1:n], y[i] <= data.pmax[i] * x[i])
    @constraint(model, eq_min_power[i in 1:n], y[i] >= data.pmin[i] * x[i])
    @constraint(model, eq_demand, sum(y[i] for i in 1:n) == data.demand)
    return model
end;

At this point, we can already use JuMP and any mixed-integer linear programming solver to find optimal solutions to any instance of this problem. To illustrate this, let us solve a small instance with three generators, using SCIP:

In [5]:
using SCIP

model = build_uc_model(
    UnitCommitmentData(
        demand = 100.0,
        pmin = [10, 20, 30],
        pmax = [50, 60, 70],
        cfix = [700, 600, 500],
        cvar = [1.5, 2.0, 2.5],
    )
)

scip = optimizer_with_attributes(SCIP.Optimizer, "limits/gap" => 1e-4)
set_optimizer(model, scip)
set_silent(model)
optimize!(model)

println("obj = ", objective_value(model))
println("  x = ", round.(value.(model[:x])))
println("  y = ", round.(value.(model[:y]), digits=2));

obj = 1320.0
  x = [0.0, 1.0, 1.0]
  y = [0.0, 60.0, 40.0]


Running the code above, we found that the optimal solution for our small problem instance costs \$1320. It is achieve by keeping generators 2 and 3 online and producing, respectively, 60 MW and 40 MW of power.

## Generating training data

Although SCIP could solve the small example above in a fraction of a second, it gets slower for larger and more complex versions of the problem. If this is a problem that needs to be solved frequently, as it is often the case in practice, it could make sense to spend some time upfront generating a **trained** version of SCIP, which can solve new instances (similar to the ones it was trained on) faster.

In the following, we will use MIPLearn to train machine learning models that can be used to accelerate SCIP's performance on a particular set of instances. More specifically, MIPLearn will train a model that is able to predict the optimal solution for instances that follow a given probability distribution, then it will provide this predicted solution to SCIP as a warm start.

Before we can train the model, we need to collect training data by solving a large number of instances. In real-world situations, we may construct these training instances based on historical data. In this tutorial, we will construct them using a random instance generator:

In [6]:
using Distributions
using Random

function random_uc_data(; samples::Int, n::Int, seed=42)
    Random.seed!(seed)
    pmin = rand(Uniform(100, 500.0), n)
    pmax = pmin .* rand(Uniform(2.0, 2.5), n)
    cfix = pmin .* rand(Uniform(100.0, 125.0), n)
    cvar = rand(Uniform(1.25, 1.5), n)
    return [
        UnitCommitmentData(;
            pmin,
            pmax,
            cfix,
            cvar,
            demand = sum(pmax) * rand(Uniform(0.5, 0.75)),
        )
        for i in 1:samples
    ]
end;

In this example, for simplicity, only the demands change from one instance to the next. We could also have made the prices and the production limits random. The more randomization we have in the training data, however, the more challenging it is for the machine learning models to learn solution patterns.

Now we generate 100 instances of this problem, each one with 1,000 generators. We will use the first 90 instances for training, and the remaining 10 instances to evaluate SCIP's performance.

In [7]:
data = random_uc_data(samples=100, n=1000);
train_data = data[1:90]
test_data = data[91:100];

Next, we write these data structures to individual files. MIPLearn uses files during the training process because, for large-scale optimization problems, it is often impractical to hold the entire training data, as well as the concrete JuMP models, in memory. Files also make it much easier to solve multiple instances simultaneously, potentially even on multiple machines. We will cover parallel and distributed computing in a future tutorial.

The code below generates the files `uc/train/000001.jld2`, `uc/train/000002.jld2`, etc., which contain the input data in [JLD2 format](https://github.com/JuliaIO/JLD2.jl).

In [8]:
using MIPLearn
MIPLearn.save(data[1:90], "uc/train/")
MIPLearn.save(data[91:100], "uc/test/")

using Glob
train_files = glob("uc/train/*.jld2")
test_files = glob("uc/test/*.jld2");

Finally, we use `MIPLearn.LearningSolver` and `MIPLearn.solve!` to solve all the training instances. `LearningSolver` is the main component provided by MIPLearn, which integrates MIP solvers and ML. The `solve!` function can be used to solve either one or multiple instances, and requires: (i) the list of files containing the training data; and (ii) the function that converts the data structure into a concrete JuMP model:

In [9]:
using Glob
solver = LearningSolver(scip)
@time solve!(solver, train_files, build_uc_model);

105.491488 seconds (94.05 M allocations: 3.632 GiB, 1.17% gc time, 0.56% compilation time)


The macro `@time` shows us how long did the code take to run. We can see that SCIP was able to solve all training instances in about 2 minutes. The solutions, and other useful training data, are stored by MIPLearn in `.h5` files, stored side-by-side with the original `.jld2` files.

## Solving new instances

With training data in hand, we can now fit the ML models using `MIPLearn.fit!`, then solve the test instances with `MIPLearn.solve!`, as shown below:

In [10]:
solver_ml = LearningSolver(scip)
fit!(solver_ml, train_files, build_uc_model)
@time solve!(solver_ml, test_files, build_uc_model);

  5.876161 seconds (9.35 M allocations: 335.718 MiB, 1.57% gc time)


The trained MIP solver was able to solve all test instances in about 6 seconds. To see that ML is being helpful here, let us repeat the code above, but remove the `fit!` line:

In [11]:
solver_baseline = LearningSolver(scip)
@time solve!(solver_baseline, test_files, build_uc_model);

 10.308870 seconds (8.19 M allocations: 279.036 MiB, 0.45% gc time)


Without the help of the ML models, SCIP took around 10 seconds to solve the same test instances.

<div class="alert alert-info">
Note
    
Note that is is not necessary to specify what ML models to use. MIPLearn, by default, will try a number of classical ML models and will choose the one that performs the best, based on k-fold cross validation. MIPLearn is also able to automatically collect features based on the MIP formulation of the problem and the solution to the LP relaxation, among other things, so it does not require handcrafted features. If you do want to customize the models and features, however, that is also possible, as we will see in a later tutorial.
</div>

## Understanding the acceleration

Let us go a bit deeper and try to understand how exactly did MIPLearn accelerate SCIP's performance. First, we are going to solve one of the test instances again, using the trained solver, but this time using the `tee=true` parameter, so that we can see SCIP's log:

In [12]:
solve!(solver_ml, test_files[1], build_uc_model, tee=true);

presolving:
(round 1, fast)       861 del vars, 861 del conss, 0 add conss, 2000 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       861 del vars, 1722 del conss, 0 add conss, 2000 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       862 del vars, 1722 del conss, 0 add conss, 2000 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (4 rounds: 4 fast, 1 medium, 1 exhaustive):
 862 deleted vars, 1722 deleted constraints, 0 added constraints, 2000 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 1138 variables (0 bin, 0 int, 0 impl, 1138 cont) and 279 constraints
    279 constraints of type <linear>
Presolving Time: 0.03

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
* 0.0s|     1 |     0 |   203 |     - |    LP  |   0

The log above is quite complicated if you have never seen it before, but the important line is the one starting with `feasible solution found [...] objective value 1.705169e+07`. This line indicates that MIPLearn was able to construct a warm start with value `1.705169e+07`. Using this warm start, SCIP then used the branch-and-cut method to either prove its optimality or to find an even better solution. Very quickly, however, SCIP proved that the solution produced by MIPLearn was indeed optimal. It was able to do this without generating a single cutting plane or running any other heuristics; it could tell the optimality by the root LP relaxation alone, which was very fast. 

Let us now repeat the process, but using the untrained solver this time:

In [13]:
solve!(solver_baseline, test_files[1], build_uc_model, tee=true);

presolving:
(round 1, fast)       861 del vars, 861 del conss, 0 add conss, 2000 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       861 del vars, 1722 del conss, 0 add conss, 2000 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       862 del vars, 1722 del conss, 0 add conss, 2000 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (4 rounds: 4 fast, 1 medium, 1 exhaustive):
 862 deleted vars, 1722 deleted constraints, 0 added constraints, 2000 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 1138 variables (0 bin, 0 int, 0 impl, 1138 cont) and 279 constraints
    279 constraints of type <linear>
Presolving Time: 0.03

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
* 0.0s|     1 |     0 |   203 |     - |    LP  |   0

In this log file, notice how the previous line about warm starts is missing. Since no warm starts were provided, SCIP had to find an initial solution using its own internal heuristics, which are not specifically tailored for this problem. The initial solution found by SCIP's heuristics has value `2.335200e+07`, which is significantly worse than the one constructed by MIPLearn. SCIP then proceeded to improve this solution, by generating cutting planes and repeatedly running additional primal heuristics. In the end, it was able to find the optimal solution, as expected, but it took longer.

In summary, MIPLearn accelerated the solution process by constructing a high-quality initial solution. In the following tutorials, we will see other strategies that MIPLearn can use to accelerate MIP performance, besides warm starts.

## Accessing the solution

In the example above, we used `MIPLearn.solve!` together with data files to solve both the training and the test instances. The solutions were saved to a `.h5` files in the train/test folders, and could be retrieved by reading theses files, but that is not very convenient. In this section we will use an easier method.

We can use the function `MIPLearn.load!` to obtain a regular JuMP model:

In [14]:
model = MIPLearn.load("uc/test/000001.jld2", build_uc_model)

A JuMP Model
Minimization problem with:
Variables: 2000
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 1000 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 1000 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1000 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 1000 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: eq_demand, eq_max_power, eq_min_power, x, y

We can then solve this model as before, with `MIPLearn.solve!`:

In [15]:
solve!(solver_ml, model)
println("obj = ", objective_value(model))
println("  x = ", round.(value.(model[:x][1:10])))
println("  y = ", round.(value.(model[:y][1:10]), digits=2))

obj = 1.7051217395548128e7
  x = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
  y = [767.11, 646.61, 230.28, 365.46, 1150.99, 1103.36, 0.0, 0.0, 0.0, 0.0]
